lstm2

과정 요약:<br> 
feature: season, day of week, week number, sales <br> 
feature engineering: 이상점 제거+ normalize+ denormalize

결과요약: RMSE: 0.24234837293624878 

상세과정:
주중(1)/주말(2) + 겨울(1)봄(2)여름(3)가을(4) // 
이미 lstm이라는 것이 sequence 개념이 있으므로 시간축(1~397)를 feature로 설정하는 것은 의미가 없을 듯 하여 LSTM 시도2에서는 제외함

추가의견) 개인적으로 이상점을 제거한 후 normalize를 하면 rmse가 커질 수 밖에 없다고 생각함. 다음시도(lstm3)은 이상점을 제거하지 않고 normalize를 하고, lstm4에서는 이상점을 제거하고 normalize를 하지 않는 것을 시도하겠다.

추가의견2) 처음에는 denormalize과정 없이 rmse를 구했는데, 이것보다는 denormalize를 한 후 rmse를 구하여 모델간 비교를 하는 것이 더 적절한 것 같다. normalize는 어디까지나 변환이니 항상 변환을 할때는 역변환을 하여 원본 데이터 형태m로 생각하는 것이 덜 헷갈리고, 결과값에 대한 더욱 직관적인 이해가 가능할 것 같다. 그리고, 여러 형태의 data transformation이 있는데 normalize 가장 마지막 단계의 data transformation인 것 같다. 다른 로그나 루트 변환을 한 후 normalization을 하고, 그리고 예측값이 모델을 통해 생성되면 바로 denormalize하고...예측(뉴럴) 모델의 input직전과 output직후.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime   

set seednumber(7 or 77)

In [2]:
tf.set_random_seed(77)

# 1. DATA 준비

In [3]:
columns=['date','sales']

txs=pd.read_table('./lstmData/lstmPrac2.csv', sep=',',header=None,names=columns )
txs.info()
txs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
date     397 non-null object
sales    397 non-null int64
dtypes: int64(1), object(1)
memory usage: 6.3+ KB


,date,sales
0,2016-01-01,34
1,2016-01-02,41
2,2016-01-03,54
3,2016-01-04,41
4,2016-01-05,35


## 1.1 기본 feature

'ds-y'의 ds로부터 api로 얻을 수 있는 값

year, day of week, month, week number를 기본 feature로 설정

In [4]:
year = lambda x: datetime.strptime(x, "%Y-%m-%d" ).year  
day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d" ).weekday()
month = lambda x: datetime.strptime(x, "%Y-%m-%d" ).month
# please read docs on how week numbers are calculate
week_number = lambda x: datetime.strptime(x, "%Y-%m-%d" ).strftime('%V')

In [5]:
txs['year'] = txs['date'].map(year)
txs['month']=txs['date'].map(month)
txs['week_number']=txs['date'].map(week_number)
txs['day_of_week']=txs['date'].map(day_of_week)

## 1.2 추가 feature

'ds-y'의 ds로부터 api로 얻을 수 없는 값: 본 feature를 가공한 feature + ds와 무관한 feature

In [6]:
seasons = [0,0,1,1,1,2,2,2,3,3,3,0] #dec - feb is winter, then spring, summer, fall etc
season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d" ).month-1)]

In [7]:
txs['season']=txs['date'].map(season)
txs

,date,sales,year,month,week_number,day_of_week,season
0,2016-01-01,34,2016,1,53,4,0
1,2016-01-02,41,2016,1,53,5,0
2,2016-01-03,54,2016,1,53,6,0
3,2016-01-04,41,2016,1,01,0,0
4,2016-01-05,35,2016,1,01,1,0
5,2016-01-06,44,2016,1,01,2,0
6,2016-01-07,50,2016,1,01,3,0
7,2016-01-08,42,2016,1,01,4,0
8,2016-01-09,42,2016,1,01,5,0
9,2016-01-10,66,2016,1,01,6,0


pandas를 통해 구한 각 feature는 list()로 우리의 기준type인 list로 변경이 가능

## 1.3 추가 가공

### 1.3.1 y의 추가 가공

이상점 제거, normalization, bucketization 을 하여 새로운 열을 생성하는 방향으로

In [24]:
sales=list(txs['sales'])

#### 이상점 제거

이상점 제거를 위해 평균과 표준편차를 구한다. 이상점의 기준은 일단 평균+-2*sd로 설정

In [33]:
mean=np.mean(sales)
std=np.std(sales)

In [34]:
for i in range(len(sales)):
    if (sales[i]<mean-2*std or sales[i]>mean+2*std):
         sales[i]=int(mean)
        

In [35]:
sales

array([ 0.34,  0.41,  0.54,  0.41,  0.35,  0.44,  0.5 ,  0.42,  0.42,
        0.66,  0.5 ,  0.55,  0.56,  0.53,  0.44,  0.54,  0.54,  0.5 ,
        0.4 ,  0.49,  0.28,  0.  ,  0.  ,  0.53,  0.43,  0.38,  0.55,
        0.49,  0.43,  0.49,  0.49,  0.44,  0.39,  0.52,  0.45,  0.33,
        0.43,  0.4 ,  0.46,  0.49,  0.5 ,  0.37,  0.37,  0.45,  0.48,
        0.48,  0.38,  0.6 ,  0.31,  0.35,  0.53,  0.  ,  0.62,  0.48,
        0.51,  0.49,  0.38,  0.32,  0.39,  0.35,  0.3 ,  0.36,  0.31,
        0.31,  0.44,  0.41,  0.41,  0.45,  0.46,  0.45,  0.41,  0.47,
        0.48,  0.4 ,  0.42,  0.38,  0.38,  0.45,  0.48,  0.62,  0.46,
        0.38,  0.62,  0.  ,  0.4 ,  0.45,  0.42,  0.53,  0.53,  0.56,
        0.53,  0.47,  0.61,  0.64,  0.62,  0.37,  0.65,  0.54,  0.44,
        0.46,  0.5 ,  0.43,  0.53,  0.63,  0.52,  0.02,  0.  ,  0.  ,
        0.  ,  0.59,  0.  ,  0.47,  0.44,  0.  ,  0.  ,  0.  ,  0.47,
        0.61,  0.  ,  0.35,  0.5 ,  0.58,  0.33,  0.41,  0.36,  0.64,
        0.49,  0.53,

#### Normalization

In [36]:
def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

In [37]:
originalSales=sales

In [38]:
sales=minMaxNormalizer(sales)

In [39]:
sales

array([ 0.49275355,  0.59420281,  0.78260858,  0.59420281,  0.5072463 ,
        0.63768107,  0.72463758,  0.60869556,  0.60869556,  0.9565216 ,
        0.72463758,  0.79710133,  0.81159409,  0.76811583,  0.63768107,
        0.78260858,  0.78260858,  0.72463758,  0.57971006,  0.71014482,
        0.40579704,  0.        ,  0.        ,  0.76811583,  0.62318832,
        0.55072456,  0.79710133,  0.71014482,  0.62318832,  0.71014482,
        0.71014482,  0.63768107,  0.56521731,  0.75362308,  0.65217382,
        0.4782608 ,  0.62318832,  0.57971006,  0.66666657,  0.71014482,
        0.72463758,  0.53623181,  0.53623181,  0.65217382,  0.69565207,
        0.69565207,  0.55072456,  0.86956509,  0.4492753 ,  0.5072463 ,
        0.76811583,  0.        ,  0.89855059,  0.69565207,  0.73913033,
        0.71014482,  0.55072456,  0.46376805,  0.56521731,  0.5072463 ,
        0.43478255,  0.52173905,  0.4492753 ,  0.4492753 ,  0.63768107,
        0.59420281,  0.59420281,  0.65217382,  0.66666657,  0.65

#### Denormalization

예측모델을 통해 얻은 sales결과를 denormalize시켜 기존 단위로 변환

In [32]:
def minMaxDeNormalizer(data):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier 

이 함수를 마지막 rmse구하기 전에 호출

## 1.4 합쳐서 최종 data 생성: XY

In [40]:
tempxy=[list(txs['season']),list(txs['day_of_week']),list(txs['week_number']),sales]

In [41]:
XY=np.array(tempxy).transpose().astype(np.float)

In [44]:
XY

NameError: name 'XY' is not defined

# 2. MODEL 설계

## 2.1 사용 model 설정: RNN LSTM

## 2.2 해당 model의 train parameters 설정

In [ ]:
#seq_length만큼의 전날 y_data가 들어가 output_dim의 다음날 y_data를 예측
seq_length=7
output_dim=1

#data_dim은 y값 도출을 위한 feature 가지수
data_dim=4

#hidden_dim은 정말 내 마음대로 설정
hidden_dim=10

learning_rate=0.01
iterations=500

## 2.3 사용 model, train parameter에 맞추어 dataset(XY) 변환: dataX, dataY

In [ ]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})
    print("RMSE: {}".format(rmse_val))

    # Plot predictions
    plt.plot(testY)
    plt.plot(test_predict)
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()